In [1]:
# Seccion de importacion de librerias
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# Replace 'path/to/serviceAccountKey.json' with the path to your downloaded service account JSON file.
cred = credentials.Certificate('credentials/serviceAccountKey.json')

# Initialize the Firebase app with the credentials.
firebase_admin.initialize_app(cred)

# Initialize Firestore DB
db = firestore.client()

In [3]:
# Acceder a la colección 'user_actions'
user_actions_ref = db.collection('user_actions')

# Obtener todos los documentos en la colección
docs = user_actions_ref.stream()

In [4]:
# Por cada dato del documento, la llave será el nombre de la columna y el valor será el valor del dato
# Pasarlo a df
data = []
for doc in docs:
    data.append(doc.to_dict())
df = pd.DataFrame(data)

In [5]:
df.head(10)

,app,date,user_id,action
0,swift,2024-09-10 05:00:00.001000+00:00,s.torresr20@gmail.com,contact
1,swift,2024-09-30 18:17:18.409000+00:00,arancru@gmail.com,filter
2,swift,2024-10-02 20:17:18.591994+00:00,arancru@gmail.com,filter
3,swift,2024-10-02 20:17:34.472924+00:00,arancru@gmail.com,filter
4,flutter,2024-09-29 22:28:23.366000+00:00,pauladaza600@gmail.com,filter
5,flutter,2024-09-30 14:05:41.976000+00:00,pauladaza600@gmail.com,filter
6,flutter,2024-10-02 05:28:23.886276+00:00,pauladaza600@gmail.com,filter
7,flutter,2024-10-02 05:28:49.345806+00:00,pauladaza600@gmail.com,filter
8,flutter,2024-10-02 05:29:02.137969+00:00,pauladaza600@gmail.com,filter
9,swift,2024-09-28 15:15:15.112000+00:00,s.torresr20@gmail.com,filter


In [6]:
# Convertir la columna 'date' a datetime sin zona horaria
df['date'] = df['date'].dt.tz_localize(None)

In [7]:
# Extraer en un df solo las filas que tengan como valor de action 'contact'
contact_df = df[df['action'] == 'contact']

# Por cada user ID dejar solo el valor que tenga la fecha mas actual
contact_df = contact_df.sort_values('date', ascending=False).drop_duplicates('user_id')

# Calcular los dias que han pasado desde esa fecha hasta hoy
contact_df['days_since_contact'] = (pd.Timestamp.now() - contact_df['date']).dt.days

# Si el valor es -1 cambiarlo por 0
contact_df['days_since_contact'] = contact_df['days_since_contact'].apply(lambda x: 0 if x == -1 else x)

In [8]:
contact_df.head(10)

,app,date,user_id,action,days_since_contact
32,swift,2024-10-02 20:17:53.613562,arancru@gmail.com,contact,0
46,swift,2024-10-02 17:53:20.133239,sofia.torres@plurall.com,contact,0
37,flutter,2024-10-02 05:28:14.283664,pauladaza600@gmail.com,contact,0
55,swift,2024-10-02 05:04:16.855969,tamaiothais@gmail.com,contact,0
44,swift,2024-09-26 22:17:31.856000,s.torresr20@gmail.com,contact,5


In [9]:
# Agregar la columna 'days_since_contact' al df original teniendo en cuenta el user_id
df = df.merge(contact_df[['user_id', 'days_since_contact']], on='user_id', how='left')

# Cualquier valor vacío en 'days_since_contact' cambiarlo por "NA"
df['days_since_contact'] = df['days_since_contact'].fillna('NA')

In [10]:
df.head(10)

,app,date,user_id,action,days_since_contact
0,swift,2024-09-10 05:00:00.001000,s.torresr20@gmail.com,contact,5.0
1,swift,2024-09-30 18:17:18.409000,arancru@gmail.com,filter,0.0
2,swift,2024-10-02 20:17:18.591994,arancru@gmail.com,filter,0.0
3,swift,2024-10-02 20:17:34.472924,arancru@gmail.com,filter,0.0
4,flutter,2024-09-29 22:28:23.366000,pauladaza600@gmail.com,filter,0.0
5,flutter,2024-09-30 14:05:41.976000,pauladaza600@gmail.com,filter,0.0
6,flutter,2024-10-02 05:28:23.886276,pauladaza600@gmail.com,filter,0.0
7,flutter,2024-10-02 05:28:49.345806,pauladaza600@gmail.com,filter,0.0
8,flutter,2024-10-02 05:29:02.137969,pauladaza600@gmail.com,filter,0.0
9,swift,2024-09-28 15:15:15.112000,s.torresr20@gmail.com,filter,5.0


In [11]:
# Acceder a la colección 'properties'
properties = db.collection('properties')

# Acceder a la colección 'offers'
offers = db.collection('offers')

# Obtener todos los documentos en la colección
docs_properties = properties.stream()
docs_offers = offers.stream()

In [12]:
# Crear una lista para almacenar los datos de cada propiedad
properties_data = []

# Recorrer cada documento en la colección 'properties'
for doc in docs_properties:
    # Extraer el contenido del documento como diccionario
    properties_dict = doc.to_dict()
    
    # Cada documento tiene un diccionario con múltiples propiedades, extraer cada propiedad
    for property_id, property_data in properties_dict.items():
        # Agregar el ID de la propiedad al diccionario de datos
        property_data['property_id'] = property_id
        # Añadir el diccionario de cada propiedad a la lista
        properties_data.append(property_data)

# Convertir la lista a un DataFrame de pandas
df_properties = pd.DataFrame(properties_data)

# Lo mismo para offers
offers_data = []

for doc in docs_offers:
    offers_dict = doc.to_dict()
    
    for offer_id, offer_data in offers_dict.items():
        offer_data['offer_id'] = offer_id
        offers_data.append(offer_data)

df_offers = pd.DataFrame(offers_data)

In [13]:
df_offers.head(10)

,price_per_month,initial_date,only_andes,final_date,views,num_baths,roommates,id_property,is_active,num_beds,user_id,num_rooms,type,offer_id
0,2500000.0,2024-12-01 14:27:00+00:00,False,2025-01-31 14:27:00+00:00,0,1,0,3,True,1,sofitor45@gmail.com,1,entire_place,4
1,1000000.0,2024-12-04 17:55:00+00:00,False,2025-01-01 17:55:00+00:00,0,1,2,5,True,2,sofitor45@gmail.com,1,a_room,6
2,1800000.0,2025-01-20 17:36:00+00:00,False,2025-06-01 17:36:00+00:00,1,1,0,4,True,1,sofitor45@gmail.com,1,a_room,5
3,3000000.0,2025-02-02 05:00:00.848000+00:00,False,2025-04-01 05:00:00.592000+00:00,45,5,2,2,True,2,sofitor45@gmail.com,2,a_room,2
4,1500000.0,2025-01-25 05:00:00.625000+00:00,True,2025-06-25 05:00:00.431000+00:00,13,1,0,1,True,4,arancru@gmail.com,4,entire_place,1


In [14]:
df_properties.head(10)

,description,photos,complex_name,address,title,location,minutes_from_campus,property_id
0,I’m going on vacations so I want to sublet my ...,"[sofitor45@gmail.com_3_1.jpg, sofitor45@gmail....",Big apartment in city U,Cra 3 # 16a - 54,Big apartment in city U,[],5,3
1,I want to sublet a bedroom in a little apartme...,"[sofitor45@gmail.com_4_1.jpg, sofitor45@gmail....",A Bedroom in the spot,Cra 5 # 20 - 32,A Bedroom in the spot,[],7,4
2,This spacious apartment in City U is shared wi...,"[apartment_image.jpg, cityu2.jpg]",City_U,"Ac. 19 #2a - 10, Bogotá",Apartment - T2 - 1102,<google.cloud.firestore_v1._helpers.GeoPoint o...,5,1
3,,"[sofitor45@gmail.com_5_1.jpg, sofitor45@gmail....",Bed in CentralX,Central X cra 3,Bed in CentralX,[],5,5
4,,"[livingx.jpg, living2.jpg]",Santa_fe,Calle 20 # 4 -34,House 5BR 1B,<google.cloud.firestore_v1._helpers.GeoPoint o...,10,2


In [15]:
# Rename en df_offers para que la columna 'id_property' sea 'property_id'
df_offers.rename(columns={'id_property': 'property_id'}, inplace = True)

# Asegúrate de que 'property_id' esté presente y sea consistente en ambos DataFrames
df_offers['property_id'] = df_offers['property_id'].astype(str)
df_properties['property_id'] = df_properties['property_id'].astype(str)

# Realizar el merge de df_offers con df_properties para agregar la descripción de la propiedad a la oferta
# 'property_id' será la columna de enlace
df_merged = df_offers.merge(df_properties[['property_id', 'description']], on='property_id', how='left')

# Teniendo en cuenta el user_id, agregar la columna 'title' del df_properties
df_merged = df_merged.merge(df_properties[['property_id', 'title']], on='property_id', how='left')

In [16]:
df_merged.head(10)

,price_per_month,initial_date,only_andes,final_date,views,num_baths,roommates,property_id,is_active,num_beds,user_id,num_rooms,type,offer_id,description,title
0,2500000.0,2024-12-01 14:27:00+00:00,False,2025-01-31 14:27:00+00:00,0,1,0,3,True,1,sofitor45@gmail.com,1,entire_place,4,I’m going on vacations so I want to sublet my ...,Big apartment in city U
1,1000000.0,2024-12-04 17:55:00+00:00,False,2025-01-01 17:55:00+00:00,0,1,2,5,True,2,sofitor45@gmail.com,1,a_room,6,,Bed in CentralX
2,1800000.0,2025-01-20 17:36:00+00:00,False,2025-06-01 17:36:00+00:00,1,1,0,4,True,1,sofitor45@gmail.com,1,a_room,5,I want to sublet a bedroom in a little apartme...,A Bedroom in the spot
3,3000000.0,2025-02-02 05:00:00.848000+00:00,False,2025-04-01 05:00:00.592000+00:00,45,5,2,2,True,2,sofitor45@gmail.com,2,a_room,2,,House 5BR 1B
4,1500000.0,2025-01-25 05:00:00.625000+00:00,True,2025-06-25 05:00:00.431000+00:00,13,1,0,1,True,4,arancru@gmail.com,4,entire_place,1,This spacious apartment in City U is shared wi...,Apartment - T2 - 1102


In [17]:
# Solo necesito el user_id, la description, views, property_id y title
df_final = df_merged[['user_id', 'description', 'views', 'property_id', 'title']]

# Rename de user_id a landlord, description a descripcion, views a vistas, property_id a property
df_final.rename(columns={'user_id': 'landlord', 'description': 'descripcion', 'views': 'vistas', 'property_id': 'property'}, inplace = True)

/var/folders/57/132bv7p52t3c33gn6rjfp63h0000gn/T/ipykernel_8734/2241412653.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(columns={'user_id': 'landlord', 'description': 'descripcion', 'views': 'vistas', 'property_id': 'property'}, inplace = True)


In [18]:
# Reemplazar las descripciones vacías con True y las llenas con False
df_final['descripcion'] = df_final['descripcion'].apply(lambda x: "Empty" if x == "" else "Complete")

/var/folders/57/132bv7p52t3c33gn6rjfp63h0000gn/T/ipykernel_8734/3583796318.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['descripcion'] = df_final['descripcion'].apply(lambda x: "Empty" if x == "" else "Complete")


In [19]:
df.head(10)

,app,date,user_id,action,days_since_contact
0,swift,2024-09-10 05:00:00.001000,s.torresr20@gmail.com,contact,5.0
1,swift,2024-09-30 18:17:18.409000,arancru@gmail.com,filter,0.0
2,swift,2024-10-02 20:17:18.591994,arancru@gmail.com,filter,0.0
3,swift,2024-10-02 20:17:34.472924,arancru@gmail.com,filter,0.0
4,flutter,2024-09-29 22:28:23.366000,pauladaza600@gmail.com,filter,0.0
5,flutter,2024-09-30 14:05:41.976000,pauladaza600@gmail.com,filter,0.0
6,flutter,2024-10-02 05:28:23.886276,pauladaza600@gmail.com,filter,0.0
7,flutter,2024-10-02 05:28:49.345806,pauladaza600@gmail.com,filter,0.0
8,flutter,2024-10-02 05:29:02.137969,pauladaza600@gmail.com,filter,0.0
9,swift,2024-09-28 15:15:15.112000,s.torresr20@gmail.com,filter,5.0


In [20]:
# Guardar tambien como dos csv serparados por ,
df_final.to_csv('data_sources/ofertas.csv', index=False, sep=',')
df.to_csv('data_sources/user_actions.csv', index=False, sep=',')

In [21]:
# Definir el alcance de las credenciales de acceso a Google Sheets
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Autorizar el cliente de gspread con las credenciales del archivo JSON
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials/client_secret.json', scope)
client = gspread.authorize(credentials)

# Abrir la hoja de cálculo de Google Sheets
spreadsheet = client.open('analyticsBQ')

In [22]:
# Crear una nueva hoja si no existe
def get_or_create_worksheet(spreadsheet, sheet_name, rows=100, cols=20):
    try:
        # Intentar obtener la hoja
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        # Crear la hoja si no existe
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=rows, cols=cols)
    return worksheet

# Obtener o crear la hoja "User Actions"
worksheet_user_actions = get_or_create_worksheet(spreadsheet, "User Actions")

# Obtener o crear la hoja "Ofertas"
worksheet_ofertas = get_or_create_worksheet(spreadsheet, "Ofertas")

In [23]:
# Leer y subir 'ofertas.csv' a la hoja "Ofertas"
df_ofertas = pd.read_csv('data_sources/ofertas.csv')
df_ofertas = df_ofertas.replace([float('inf'), float('-inf'), None], 0)  # Reemplazar valores infinitos con 0
df_ofertas = df_ofertas.fillna('')  # Reemplazar NaN con cadenas vacías
worksheet_ofertas.clear()  # Limpiar la hoja antes de subir los nuevos datos
worksheet_ofertas.update([df_ofertas.columns.values.tolist()] + df_ofertas.values.tolist())  # Subir el contenido

# Leer y subir 'user_actions.csv' a la hoja "User Actions"
df_user_actions = pd.read_csv('data_sources/user_actions.csv')
df_user_actions = df_user_actions.replace([float('inf'), float('-inf'), None], 0)  # Reemplazar valores infinitos con 0
df_user_actions = df_user_actions.fillna('')  # Reemplazar NaN con cadenas vacías
worksheet_user_actions.clear()  # Limpiar la hoja antes de subir los nuevos datos
worksheet_user_actions.update([df_user_actions.columns.values.tolist()] + df_user_actions.values.tolist())  # Subir el contenido

print("Datos subidos exitosamente a las hojas 'User Actions' y 'Ofertas'")

Datos subidos exitosamente a las hojas 'User Actions' y 'Ofertas'
